# 源代码

本笔记本介绍如何使用一种特殊的语言解析方法加载源代码文件：代码中的每个顶级函数和类都将被加载到单独的文档中。任何已加载函数和类之外的剩余顶级代码都将被加载到另一个单独的文档中。

这种方法有可能提高 QA 模型处理源代码的准确性。

支持的代码解析语言包括：

- C (*)
- C++ (*)
- C# (*)
- COBOL
- Elixir
- Go (*)
- Java (*)
- JavaScript (需要包 `esprima`)
- Kotlin (*)
- Lua (*)
- Perl (*)
- Python
- Ruby (*)
- Rust (*)
- Scala (*)
- TypeScript (*)

标记为 (*) 的项目需要 `tree_sitter` 和 `tree_sitter_languages` 包。
使用 `tree_sitter` 可以轻松地为其他语言添加支持，
尽管目前这需要修改 LangChain。

可以配置用于解析的语言，以及基于语法激活拆分的最小行数。

如果未明确指定语言，`LanguageParser` 将从文件名扩展（如果存在）中推断出一种语言。

In [ ]:
%pip install -qU esprima esprima tree_sitter tree_sitter_languages

In [1]:
import warnings

warnings.filterwarnings("ignore")
from pprint import pprint

from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language

In [2]:
loader = GenericLoader.from_filesystem(
    "./example_data/source_code",
    glob="*",
    suffixes=[".py", ".js"],
    parser=LanguageParser(),
)
docs = loader.load()

In [3]:
len(docs)

6

In [4]:
for document in docs:
    pprint(document.metadata)

{'content_type': 'functions_classes',
 'language': <Language.PYTHON: 'python'>,
 'source': 'example_data/source_code/example.py'}
{'content_type': 'functions_classes',
 'language': <Language.PYTHON: 'python'>,
 'source': 'example_data/source_code/example.py'}
{'content_type': 'simplified_code',
 'language': <Language.PYTHON: 'python'>,
 'source': 'example_data/source_code/example.py'}
{'content_type': 'functions_classes',
 'language': <Language.JS: 'js'>,
 'source': 'example_data/source_code/example.js'}
{'content_type': 'functions_classes',
 'language': <Language.JS: 'js'>,
 'source': 'example_data/source_code/example.js'}
{'content_type': 'simplified_code',
 'language': <Language.JS: 'js'>,
 'source': 'example_data/source_code/example.js'}


In [7]:
print("\n\n--8<--\n\n".join([document.page_content for document in docs]))

class MyClass:
    def __init__(self, name):
        self.name = name

    def greet(self):
        print(f"Hello, {self.name}!")

--8<--

def main():
    name = input("Enter your name: ")
    obj = MyClass(name)
    obj.greet()

--8<--

# Code for: class MyClass:


# Code for: def main():


if __name__ == "__main__":
    main()

--8<--

class MyClass {
  constructor(name) {
    this.name = name;
  }

  greet() {
    console.log(`Hello, ${this.name}!`);
  }
}

--8<--

function main() {
  const name = prompt("Enter your name:");
  const obj = new MyClass(name);
  obj.greet();
}

--8<--

// Code for: class MyClass {

// Code for: function main() {

main();


解析器可以针对小型文件禁用。

参数 `parser_threshold` 指示源文件必须具有的最小行数，才能使用解析器进行分段。

In [8]:
loader = GenericLoader.from_filesystem(
    "./example_data/source_code",
    glob="*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=1000),
)
docs = loader.load()

In [9]:
len(docs)

1

In [10]:
print(docs[0].page_content)

class MyClass:
    def __init__(self, name):
        self.name = name

    def greet(self):
        print(f"Hello, {self.name}!")


def main():
    name = input("Enter your name: ")
    obj = MyClass(name)
    obj.greet()


if __name__ == "__main__":
    main()



## 分割

对于那些过大的函数、类或脚本，可能还需要进行额外的分割。

In [11]:
loader = GenericLoader.from_filesystem(
    "./example_data/source_code",
    glob="*",
    suffixes=[".js"],
    parser=LanguageParser(language=Language.JS),
)
docs = loader.load()

In [12]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

In [13]:
js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=60, chunk_overlap=0
)

In [14]:
result = js_splitter.split_documents(docs)

In [15]:
len(result)

7

In [16]:
print("\n\n--8<--\n\n".join([document.page_content for document in result]))

class MyClass {
  constructor(name) {
    this.name = name;

--8<--

}

--8<--

greet() {
    console.log(`Hello, ${this.name}!`);
  }
}

--8<--

function main() {
  const name = prompt("Enter your name:");

--8<--

const obj = new MyClass(name);
  obj.greet();
}

--8<--

// Code for: class MyClass {

// Code for: function main() {

--8<--

main();


## 使用 Tree-sitter 模板添加语言

使用 Tree-Sitter 模板扩展语言支持涉及几个关键步骤：

1. **创建新的语言文件**：
    - 首先，在指定的目录 (langchain/libs/community/langchain_community/document_loaders/parsers/language) 中创建一个新文件。
    - 以现有语言文件（如 **`cpp.py`**）的结构和解析逻辑为模型。
    - 您还需要在 langchain 目录 (langchain/libs/langchain/langchain/document_loaders/parsers/language) 中创建一个文件。
2. **解析语言特性**：
    - 模仿 **`cpp.py`** 文件中使用的结构，并根据您要添加的语言进行调整。
    - 主要的修改在于调整 chunk query 数组以适应您正在解析的语言的语法和结构。
3. **测试语言解析器**：
    - 为了进行全面验证，请为新语言生成一个特定的测试文件。在指定的目录 (langchain/libs/community/tests/unit_tests/document_loaders/parsers/language) 中创建 **`test_language.py`**。
    - 遵循 **`test_cpp.py`** 的示例，为新语言中的解析元素建立基础测试。
4. **集成到解析器和文本分割器中**：
    - 在 **`language_parser.py`** 文件中集成您的新语言。请确保更新 LANGUAGE_EXTENSIONS 和 LANGUAGE_SEGMENTERS 以及 LanguageParser 的 docstring，以识别和处理已添加的语言。
    - 同时，请确认您的语言已包含在 **`text_splitter.py`** 中的 Language 类里，以进行正确的解析。

通过遵循这些步骤并确保全面的测试和集成，您将成功地使用 Tree-Sitter 模板扩展语言支持。

祝您好运！